<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Init-Dependencies" data-toc-modified-id="Init-Dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Init Dependencies</a></span></li><li><span><a href="#Set-Static-Variables" data-toc-modified-id="Set-Static-Variables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set Static Variables</a></span></li><li><span><a href="#Set-Authentication" data-toc-modified-id="Set-Authentication-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set Authentication</a></span></li><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Helper Functions</a></span></li><li><span><a href="#Begin-Transformations" data-toc-modified-id="Begin-Transformations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Begin Transformations</a></span></li><li><span><a href="#Update-Result-Framework" data-toc-modified-id="Update-Result-Framework-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Update Result Framework</a></span></li></ul></div>

## Init Dependencies

In [1]:
import requests
import os
import pandas as pd
from datetime import datetime

## Set Static Variables

In [2]:
URL = 'https://rsr.akvo.org/rest/v1/'
PROJECT_ID = '7283'
RSR_TOKEN = os.environ['RSR_TOKEN']
FMT = '/?format=json&limit=1'
FMT100 = '/?format=json&limit=100'

## Set Authentication

In [3]:
headers = {
    'content-type': 'application/json',
    'Authorization': RSR_TOKEN
}

## Helper Functions

In [4]:
def get_response(endpoint, param, value):
    uri = '{}{}{}&{}={}'.format(URL, endpoint, FMT100, param, value)
    print(get_time() + ' Fetching - ' + uri)
    data = requests.get(uri, headers=headers)
    data = data.json()
    return data

In [5]:
def get_time():
    now = datetime.now().time().strftime("%H:%M:%S")
    return now

In [6]:
def get_sibling_id(x):
    for k,v in x.items():
        return k

In [7]:
def get_report_type(ps,pe):
    rt = {'is_yearly':False}
    psm = ps.split('-')[1]
    pem = pe.split('-')[1]
    if psm == '01' and pem == '12':
        rt = {'is_yearly':True}
    if psm == '01' and pem == '01':
        rt = {'is_yearly':True}
    return rt

In [8]:
def get_dimension_country(dv):
    dp = dv['value'].split(' - ')
    dv = {}
    if dp[0].lower() in ['zambia','malawi','mozambique']:
        dv.update({
            'commodity':'',
            'country':dp[0],
            'has_commodity':False,
            'has_country':True
        })
    else:
        dv.update({
            'commodity':dp[0],
            'country':'',
            'has_commodity':False,
            'has_country':True
        })
    if len(dp) == 2:
        dv.update({
            'commodity':dp[0],
            'country':dp[1],
            'has_commodity':True,
            'has_country':True
        })
    return dv

## Begin Transformations

In [9]:
results_framework = get_response('results_framework','project',PROJECT_ID)['results']

05:47:53 Fetching - https://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=7283


In [10]:
indicators = []
periods = []
dimension_names = []
dimension_values = []
for result_framework in results_framework:
    rf_id = {'result':result_framework['id']}
    for indicator in result_framework['indicators']:
        for period in indicator['periods']:
            is_yearly = get_report_type(period['period_start'],period['period_end'])
            period.update(is_yearly)
            period.update(rf_id)
            periods.append(period)
        del indicator['periods']
        for dimension_name in indicator['dimension_names']:
            for dimension_value in dimension_name['values']:
                dimension_value.update(rf_id)
                dimension_update = get_dimension_country(dimension_value)
                dimension_value.update(dimension_update)
                dimension_values.append(dimension_value)
            del dimension_name['values']
            dimension_name.update(rf_id)
            dimension_names.append(dimension_name)
        del indicator['dimension_names']
        indicators.append(indicator)

## Update Result Framework

In [11]:
periods_df = pd.DataFrame(periods)
periods_df = periods_df.groupby(['is_yearly','result']).size().to_frame('size').reset_index().to_dict('records')

In [12]:
reports_annual = []
reports_semester = []
reports_both = []

In [13]:
for period_df in periods_df:
    if period_df['is_yearly']:
        reports_annual.append(period_df['result'])
    else:
        reports_semester.append(period_df['result'])
for y in reports_annual:
    for s in reports_semester:
        if y == s:
            reports_both.append(y)
for m in reports_both:
    reports_annual.remove(m)
    reports_semester.remove(m)

In [15]:
reports_both

[30243, 30307, 30308, 30449]

In [16]:
results_framework_new = []
for rf in results_framework:
    report_type = 'both'
    if rf['id'] in reports_annual:
        report_type = 'annual'
    if rf['id'] in reports_semester:
        report_type = 'semeseter'
    rf.update({'report_type':report_type})
    try:
        child_project = get_sibling_id(rf['child_projects'])
        rf.update({'child_projects': child_project})
    except:
        rf.update({'child_projects': None})
    try:
        parent_project = get_sibling_id(rf['parent_project'])
        rf.update({'parent_project': parent_project})
    except:
        rf.update({'parent_project': None})
    del rf['indicators']
    results_framework_new.append(rf)

In [17]:
response = {
    'results_framework':results_framework_new,
    'indicators':indicators,
    'periods':periods,
    'dimension_names':dimension_names,
    'dimension_values':dimension_values
}

In [19]:
pd.DataFrame(response['results_framework'])

,id,project,project_title,parent_project,child_projects,title,type,aggregation_status,description,order,parent_result,report_type
0,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
1,36599,7283,APPSA Regional - Don't change,None,7950,Intermediate Result 1: Improved collaborative ...,2,None,,1,None,annual
2,36602,7283,APPSA Regional - Don't change,None,7950,Intermediate Result 2: Improved technical capa...,2,None,,2,None,annual
3,36886,7283,APPSA Regional - Don't change,None,7950,Intermediate Result 3: Effective structures an...,2,None,,3,None,annual
4,30307,7283,APPSA Regional - Don't change,None,7282,Promising technologies identified and tested,1,None,,4,None,both
5,30238,7283,APPSA Regional - Don't change,None,7282,"New dissemination packages and tools designed,...",1,None,,5,None,semeseter
6,30243,7283,APPSA Regional - Don't change,None,7282,"Improved regional production, supply and deliv...",2,None,,6,None,both
7,30449,7283,APPSA Regional - Don't change,None,7282,Research institutes are effectively working to...,2,None,,7,None,both
8,30239,7283,APPSA Regional - Don't change,None,None,Improved infrastructure and equipment for targ...,1,None,,8,None,semeseter
9,30423,7283,APPSA Regional - Don't change,None,None,Improved management and performance systems at...,1,None,,9,None,semeseter


In [20]:
pd.DataFrame(response['indicators'])

,id,parent_indicator,children_aggregate_percentage,title,type,measure,ascending,description,baseline_year,baseline_value,baseline_comment,order,default_periods,export_to_iati,result
0,47749,None,False,PDO 1 Number of technologies that are being ma...,1,1,None,"Technology refers to the tools, methods or mac...",2013.0,0,,0.0,True,True,30308
1,47753,None,True,PDO 2 Proportion of Lead Farmers in targeted a...,1,2,None,Lead farmers represent a group of innovative a...,2013.0,0,,1.0,False,True,30308
2,47750,None,False,PDO 3 Number of technologies generated or prom...,1,1,None,“Technology generated” - innovation developed....,NaN,,,2.0,False,True,30308
3,47751,None,False,PDO 4 Number of direct project beneficiaries,1,1,None,Direct beneficiaries refers to the persons (e....,2013.0,0,,3.0,False,True,30308
4,79616,None,False,Number of collaborative research and developme...,1,1,True,,2013.0,0,,0.0,False,True,36599
5,79617,None,True,Percentage of collaborative research and devel...,1,2,True,,2013.0,0,,1.0,False,True,36599
6,80748,None,False,Total number of technologies generated,1,1,True,,2013.0,0,,2.0,False,True,36599
7,79619,None,False,Total number of improved technologies formally...,1,1,None,,2013.0,0,,3.0,False,True,36599
8,79620,None,True,Percentage of APPSA funded R&D dissemination p...,1,2,None,,2013.0,0,,4.0,False,True,36599
9,79621,None,False,Number of clients (research and advisory servi...,1,1,None,,2013.0,0,,0.0,False,True,36602


In [21]:
pd.DataFrame(response['periods'])

,id,data,parent_period,percent_accomplishment,locked,period_start,period_end,target_value,target_comment,actual_value,actual_comment,numerator,denominator,narrative,indicator,is_yearly,result
0,91039,[],None,None,False,2013-01-01,2013-12-31,,,,,None,None,,47749,True,30308
1,91043,[],None,None,True,2014-01-01,2014-12-31,,,0,,None,None,,47749,True,30308
2,91047,[],None,None,True,2015-01-01,2015-12-31,,,3.00,,None,None,,47749,True,30308
3,91051,[],None,None,True,2016-01-01,2016-12-31,,,2.00,,None,None,,47749,True,30308
4,91055,[],None,None,True,2017-01-01,2017-12-31,,,4.00,,None,None,,47749,True,30308
5,91059,[],None,None,True,2018-01-01,2018-12-31,,,4.00,,None,None,,47749,True,30308
6,91063,[],None,None,True,2019-01-01,2020-01-31,,,,,None,None,,47749,True,30308
7,91476,[],None,None,False,2013-01-01,2013-12-31,,,,,None,None,,47753,True,30308
8,91480,[],None,None,True,2014-01-01,2014-12-31,,,,,None,None,,47753,True,30308
9,91484,[],None,None,True,2015-01-01,2015-12-31,,,,,None,None,,47753,True,30308


In [56]:
d_names = pd.DataFrame(response['dimension_names'])

In [85]:
d_names = d_names.rename(columns={'name':'aggregate'})

In [86]:
d_values = pd.DataFrame(response['dimension_values'])

In [87]:
d_values = d_values.merge(d_names, how='inner', left_on='name', right_on='id')

In [88]:
d_values = d_values.groupby(['aggregate','commodity','country']).size().to_frame('size').sort_index().reset_index()

In [102]:
d_values.groupby(['aggregate','commodity','country']).first()

size
aggregate      commodity       country         
289            Cassava         Malawi       100
                               Mozambique   100
                               Zambia       100
               Legumes         Malawi       100
                               Mozambique   100
...                                         ...
maize type     OPV             Mozambique     1
                               Zambia         1
type of farmer Female farmer                  4
               Follower farmer                4
               Lead farmer                    4

[138 rows x 1 columns]